# Chennai's quest to quench its thirst

### Chennai & its water sources

Chennai also known as Madras, is the capital of the Indian state of Tamil Nadu. Located on the Coromandel Coast off the Bay of Bengal, it is the biggest cultural, economic and educational centre of south India. Population of Chennai is close to 9 million and is the 36th largest urban area by population in the world

Chennai is entirely dependent on ground water resources to meet its water needs. Ground water resources in Chennai are replenished by rain water and the city's average rainfall is 1,276 mm1.

Following are the major sources of water supply for Chennai city.

Four major reservoirs in Red Hills, Cholavaram, Poondi and Chembarambakkam
Following are the major sources of water supply for Chennai city.

1. Four major reservoirs in Red Hills, Cholavaram, Poondi and Chembarambakkam
2. Cauvery water from Veeranam lake
3. Desalination plants at Nemelli and Minjur
4. Aquifers in Neyveli, Minjur and Panchetty
5. Tamaraipakkam, Poondi and Minjur Agriculture wells
6. CMWSSB Boreweels
7. Retteri lake

The above one is also roughly the descending order in which the contribution is made to overall fresh water requirements of the city. In addition to this, people make use of borewells and private tankers for their water needs.

Chennai is facing an acute water shortage due to shortage of rainfall for the past three years (and we had one of the worst floods in history the year before that!). As a result, the water in these resources are depleting along with the groundwater level. This [video](https://www.youtube.com/watch?v=iaG7kRcSxwA&feature=youtu.be) will give an idea about the current state.

### Content
This dataset has details about the water availability in the four main reservoirs over the last 15 years.
All the measurements are in mcft (million cubic feet)

Poondi
Cholavaram
Redhills
Chembarambakkam


In this notebook, let us explore the data of different water resources available.

## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime

In [ ]:
!ls /kaggle/input/**/*

In [ ]:
import glob
csv_list = glob.glob('/kaggle/input/**/*')
csv_list

## Read the data

Firstly, we have data about the water availability in four major reservoirs that supply water to Chennai. This data spans from 2004 to 2019. All the measurements are in mcft (million cubic feet). Let us look at the top few lines.

In [ ]:
df = pd.read_csv(csv_list[0])
df['Date'] = pd.to_datetime(df.Date, format='%d-%m-%Y')
df.head()

In [ ]:
df.dtypes

In [ ]:
df.isna().sum()

## Find out and compare the water levels of the 4 major resoviours over a period of time



In [ ]:
px.line(df.melt(id_vars='Date',var_name='Resoviour',value_name='Water_Level'),
       x='Date',
       y = 'Water_Level',
       color = 'Resoviour',
       title = 'water availability in chennai'
       )

## Combine the major water reservoirs to get a better picture and note down your observations

In [ ]:
df['total']=df.drop(columns='Date').sum(axis=1)
df.head()

In [ ]:
px.line(df,
       x='Date',
       y = 'total',
       title = 'total water availability in chennai'
       )

## Rainfall Levels in Reservoir Regions

Now there are two clear facts:

There is no water in any of the major reservoirs
Reservoirs depend on rain for their replenishment.

### Next we can look at the rainfall data in these reservoir regions to analyze the rainfall months. Let us take the total monthly rainfall in these reservoir regions and plot the same.

* Read the data

* Combine the rainfall data for major reservoirs

* Plot the rainfall data

* Note down your observation

Note - Hover over the graph to see the better results

In [ ]:
rain_df = pd.read_csv(csv_list[1],parse_dates=['Date'],dayfirst=True)
rain_df.head()

In [ ]:
rain_df.dtypes

In [ ]:
rain_df.isna().sum()

## Plot the yearly rainfall data and note your observations

In [ ]:
px.line(rain_df.melt(id_vars='Date',var_name='Resoviour',value_name='Rainfall'),
       x='Date',
       y = 'Rainfall',
       color = 'Resoviour',
        facet_col = 'Resoviour',
        facet_col_wrap = 2,
       title = 'Daily rainfall in chennai'
       )

In [ ]:
rain_df['total']=rain_df.drop(columns='Date').sum(axis=1)
rain_df.head()

In [ ]:
rain_df['Yearmonth'] = pd.to_datetime(rain_df.Date.dt.year.astype(str)+rain_df.Date.dt.month.astype(str),format='%Y%m')
rain_df.head()

In [ ]:
monthly_df=rain_df.groupby('Yearmonth').total.sum().reset_index()
monthly_df

In [ ]:
rain_df.Yearmonth.value_counts()

In [ ]:
px.bar(monthly_df,
       x='Yearmonth',
       y = 'total',
       title = 'total monthly rainfall in chennai'
       )

In [ ]:
weekly = rain_df.groupby([rain_df.Date.dt.year,rain_df.Date.dt.week]).total.sum()
weekly.index = weekly.index.set_names('Week',level=1).set_names('Year',level=0)
weekly.reset_index()

In [ ]:
px.bar(weekly.reset_index(),
      x = ['Week'],
      y = 'total',
      facet_col = 'Year',
      facet_col_wrap = 4,
      title = 'weekly rainfall in chennai',
      height =1500
      )

In [ ]:
month_to_season = {1:'winter',2:'winter',3:'summer',4:'summer',5:'summer',6:'monsoon',7:'monsoon',8:'monsoon',9:'monsoon',10:'post_monsoon',11:'post_monsoon',12:'winter'}

In [ ]:
monthly_df['season'] = monthly_df.Yearmonth.dt.month.map(month_to_season)
monthly_df.head()

In [ ]:
px.bar(monthly_df,
       x='Yearmonth',
       y = 'total',
       color = 'season',
       title = ' monthly rainfall in chennai season'
       )


In [ ]:
rain_df

In [ ]:
px.bar(monthly_df.groupby(monthly_df.Yearmonth.dt.year).total.sum().reset_index(),
      x='Yearmonth',
      y='total',
      title='yearly rainfall in chennai')

In [ ]:
px.bar(monthly_df.groupby([monthly_df.Yearmonth.dt.year,'season']).total.sum().reset_index(),
      x='Yearmonth',
      y='total',
      color = 'season',
      title='yearly rainfall in chennai')

## Water shortage estimation

Since all the data is available in the public domain, we want to do some analysis and see whether we can estimate this water shortage ahead of time so as to plan for it?

First let us just take a simple step to compare the sum of water levels at the beginning of summer (Let us take February 1st of every year). This is because there will not be any replenishment till the next monsoon and the amount of water stored in the four reservoirs itself will be clear indicator of how long can the water be managed during summer and whether there should be some backup plans.

In [ ]:
df.loc[(df.Date.dt.month==3) & (df.Date.dt.day==1)].total.mean()

In [ ]:
fig = px.bar(df.loc[(df.Date.dt.month==3) & (df.Date.dt.day==1)],
      x='Date',
      y='total',
      title = 'Availabilit of water in total  at beginning of summer')
fig.add_hline(y=df.loc[(df.Date.dt.month==3) & (df.Date.dt.day==1)].total.mean())